#PNAD COVID
## Analise dos dados do PNAD (https://covid19.ibge.gov.br/pnad-covid/).
### Dicionário referência: (https://docs.google.com/spreadsheets/d/1yLM8VCWtyYSLcW4xWrtV_x3eOautvfXnPgWdrJq8egQ/edit?usp=sharing)

In [ ]:
print("Vamos flamengo!")

Vamos flamengo!


#Bibliotecas usadas


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, lower, lit, monotonically_increasing_id, last
from pyspark.sql.window import Window
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import re
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from google.colab import files

#IMPORTANDO OS DADOS


In [ ]:
def carregar_dicionarios(lista_arquivos):
    """
    Carrega e concatena dicionários da PNAD COVID.

    - lista_arquivos: lista de caminhos para arquivos .xls
    - Retorna: DataFrame único com colunas [codigo_var, nome_var, codigo_valor, descricao_valor]
    """
    dfs = []

    for arquivo in lista_arquivos:
        df_dic = pd.read_excel(
            arquivo,
            sheet_name=0,
            skiprows=3  # pula cabeçalho desnecessário
        )

        # seleciona apenas colunas relevantes (ajuste se necessário)
        df_dic = df_dic.iloc[:, [1, 3, 4, 5]]
        df_dic.columns = ["codigo_var", "nome_var", "codigo_valor", "descricao_valor"]

        # preencher valores para baixo
        df_dic["codigo_var"] = df_dic["codigo_var"].ffill()
        df_dic["nome_var"] = df_dic["nome_var"].ffill()

        # normalizar código de valor
        df_dic["codigo_valor"] = (
            df_dic["codigo_valor"]
            .astype(str)
            .str.strip()
            .str.replace(r"\.0$", "", regex=True)
            .replace("nan", pd.NA)
        )

        dfs.append(df_dic)

    # concatena tudo
    df_final = pd.concat(dfs, ignore_index=True).drop_duplicates()
    return df_final


In [ ]:
arquivos = [
    "/content/Dicionario_PNAD_COVID_092020_20220621 (1).xls",
    "/content/Dicionario_PNAD_COVID_102020_20220621.xls",
    "/content/Dicionario_PNAD_COVID_112020_20220621.xls"
]

df_dic = carregar_dicionarios(arquivos)
df_dic.head(200)


,codigo_var,nome_var,codigo_valor,descricao_valor
0,Ano,Ano de referência,<NA>,NaN
1,UF,Unidade da Federação,11,Rondônia
2,UF,Unidade da Federação,12,Acre
3,UF,Unidade da Federação,13,Amazonas
4,UF,Unidade da Federação,14,Roraima
...,...,...,...,...
195,B00110,Na semana passada teve dor nos olhos?,9,Ignorado
196,B00111,Na semana passada teve perda de cheiro ou sabor?,1,Sim
197,B00111,Na semana passada teve perda de cheiro ou sabor?,2,Não
198,B00111,Na semana passada teve perda de cheiro ou sabor?,3,Não sabe


In [ ]:
#Criando sessão spark
spark = SparkSession.builder \
    .appName("PNAD_COVID") \
    .getOrCreate()

In [ ]:
#importando dados
df = spark.read.csv(
    "/content/PNAD_COVID_*.csv",   # lê todos os CSV que batem com o padrão
    header=True,
    sep=",",
    inferSchema=True
)

#Capturo o nome dos arquivos para criar uma coluna nova com Mes/ano do dado
df = df.withColumn("input_file", F.input_file_name())
df = df.withColumn(
    "mes_referencia",
    F.regexp_extract(F.col("input_file"), r"PNAD_COVID_(\d{6})", 1)
)
df.show(5)

+----+---+-------+-------+-----+-----+-----+-----+-------+---------+-----+-----+------+------------+------------+------+----+-----+------+------+------+----+----+----+----+----+-----+-----+----+-----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+------+------+-----+----+----+-----+----+-----+------+--------+-----+------+------+-----+-----+-----+------+-------+--------+------+-------+-------+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-----+--------+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+------+------+------+------+------+-----+----+--------------------+--------------+
| Ano| UF|CAPITAL|RM_RIDE|V1008|V1012|V1013|V10

#Tratando os dados


In [ ]:
colunas = ['UF',
'V1022',
'A002',
'A003',
'B0011',
'B0012',
'B0013',
'B0014',
'B0015',
'B0016',
'B0017',
'B0018',
'B0019',
'B00110',
'B00111',
'B00112',
'B002',
'B0031',
'B007',
'B008',
'B009B',
'B0101',
'B0102',
'B0103',
'B0104',
'B0105',
'B0106',
'B011',
'C01012',
'D0051',
'E001',
'mes_referencia']

df = df.select(*[c for c in colunas if c in df.columns])

print("linhas após select:", df.count())
df.show(5)


linhas após select: 1075550
+---+-----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+----+-----+----+----+-----+-----+-----+-----+-----+-----+-----+----+--------+-----+----+--------------+
| UF|V1022|A002|A003|B0011|B0012|B0013|B0014|B0015|B0016|B0017|B0018|B0019|B00110|B00111|B00112|B002|B0031|B007|B008|B009B|B0101|B0102|B0103|B0104|B0105|B0106|B011|  C01012|D0051|E001|mes_referencia|
+---+-----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+----+-----+----+----+-----+-----+-----+-----+-----+-----+-----+----+--------+-----+----+--------------+
| 11|    1| 036|   1|    2|    2|    2|    2|    2|    2|    2|    2|    2|     2|     2|     2|NULL| NULL|   1|   2| NULL|    2|    2|    2|    2|    2|    2|   2|00001045|    1|   3|        112020|
| 11|    1| 030|   2|    2|    2|    2|    2|    2|    2|    2|    2|    2|     2|     2|     2|NULL| NULL|   2|   2| NULL|    2|    2|    2|    2|    2|    2|   2|00001045

In [ ]:
# --- 1. Criar dicionários ---
map_vars = dict(zip(df_dic["codigo_var"], df_dic["nome_var"]))

map_values = {}
for var in df_dic["codigo_var"].unique():
    sub = df_dic[df_dic["codigo_var"] == var]
    d = dict(zip(sub["codigo_valor"], sub["descricao_valor"]))
    if d:  # só se tiver categorias
        map_values[var] = d

map_vars

{'Ano': 'Ano de referência',
 'UF': 'Unidade da Federação',
 'CAPITAL': 'Capital',
 'RM_RIDE': 'Região Metropolitana e Região Administrativa Integrada\nde Desenvolvimento',
 'V1008': 'Número de seleção do domicílio',
 'V1012': 'Semana no mês',
 'V1013': 'Mês da pesquisa',
 'V1016': 'Número da entrevista no domicílio',
 'Estrato': 'Estrato',
 'UPA': 'UPA',
 'V1022': 'Situação do domicílio',
 'V1023': 'Tipo de área',
 'V1030': 'Projeção da população',
 'V1031': 'Peso do domicílio e das pessoas',
 'V1032': 'Peso do domicílio e das pessoas',
 'posest': 'Domínios de projeção',
 'A001': 'Número de ordem',
 'A001A': 'Condição no domicílio',
 'A001B1': 'Dia de nascimento',
 'A001B2': 'Mês de nascimento',
 'A001B3': 'Ano de nascimento',
 'A002': 'Idade do morador ',
 'A003': 'Sexo',
 'A004': 'Cor ou raça',
 'A005': 'Escolaridade',
 'A006': 'Frequenta escola',
 'A007': 'Na semana passada, _____ foram disponibilizadas atividades escolares para realizar em casa?',
 'A008': 'Na semana passada, em q

In [ ]:
def substituir_valores_df(df, map_values):
    """
    Substitui valores das colunas no DataFrame usando os mapeamentos de map_values
    e normaliza os nomes das colunas (sem acento, ç->c, sem espaços, sem caracteres especiais).
    """

    # 🔹 Normaliza os nomes das colunas antes de processar
    col_map = {col: normalizar_nome(col) for col in df.columns}
    df = df.toDF(*[col_map[c] for c in df.columns])

    # 🔹 Também normaliza as chaves do dicionário map_values
    new_map_values = {}
    for col, mapping in map_values.items():
        col_norm = normalizar_nome(col)
        new_map_values[col_norm] = mapping
    map_values = new_map_values

    # 🔹 Substitui os valores conforme o dicionário
    for col, mapping in map_values.items():
        if col not in df.columns:
            continue

        # Normaliza as chaves e valores para string
        mapping = {str(k).strip(): (str(v).strip() if v is not None else None)
                   for k, v in mapping.items()}

        # Normaliza a coluna para string e remove ".0"
        df = df.withColumn(col, F.trim(F.col(col).cast("string")))
        df = df.withColumn(col, F.regexp_replace(F.col(col), r"\.0$", ""))

        # Cria mapping expression
        map_items = []
        for k, v in mapping.items():
            if k.lower() in ("nan", "none", "<na>", ""):
                continue
            map_items.extend([F.lit(k), F.lit(v)])

        if not map_items:
            continue

        mapping_expr = F.create_map(*map_items)

        # Substitui valores (mantém original se não existir no mapping)
        df = df.withColumn(
            col,
            F.coalesce(
                mapping_expr.getItem(F.col(col)),
                F.col(col)
            )
        )

    return df

def normalizar_nome(nome):
    """Normaliza string para nome de coluna: remove acentos, pontuação, espaços -> _, lower."""
    if not isinstance(nome, str):
        return nome
    s = unicodedata.normalize("NFD", nome)
    s = s.encode("ascii", "ignore").decode("utf-8")
    s = s.lower().replace("ç", "c")
    s = re.sub(r"[,\?\.\-/():;]", "", s)
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"_+", "_", s)
    s = s.strip("_")
    return s

def achar_col_variantes(df, codigo, human_name=None):
    """
    Tenta localizar no df uma coluna correspondente ao codigo.
    Testa várias variantes: codigo, codigo.lower, codigo.upper, normalizar(codigo),
    human_name (e normalizada).
    Retorna o nome da coluna encontrada (ou None).
    """
    candidates = []
    candidates.append(str(codigo))
    candidates.append(str(codigo).lower())
    candidates.append(str(codigo).upper())
    candidates.append(normalizar_nome(str(codigo)))
    candidates.append(normalizar_nome(str(codigo)).lower())
    if human_name:
        candidates.append(str(human_name))
        candidates.append(str(human_name).lower())
        candidates.append(normalizar_nome(str(human_name)))
        candidates.append(normalizar_nome(str(human_name)).lower())
    # remove duplicados preservando ordem
    seen = set()
    candidates = [c for c in candidates if not (c in seen or seen.add(c))]
    cols = set(df.columns)
    for c in candidates:
        if c in cols:
            return c
    return None

def renomear_colunas_robusto(df, map_vars, normalize_to_snake=True, verbose=True):
    """
    Renomeia as colunas do df usando map_vars (codigo -> nome legivel).
    - Procura a coluna atual testando variantes do código e do nome legivel.
    - Renomeia para normalizar_nome(nome_legivel) por padrão (snake_case).
    - Retorna df atualizado, lista renomeados, lista nao_encontrados.
    """
    # garantir strings limpas no map_vars
    map_vars_clean = {str(k).strip(): str(v).strip() for k,v in map_vars.items()}
    renomeados = []
    nao_encontrados = []

    for codigo, human in map_vars_clean.items():
        found = achar_col_variantes(df, codigo, human)
        if not found:
            nao_encontrados.append(codigo)
            continue

        # novo nome final
        novo = normalizar_nome(human) if normalize_to_snake else human

        # se já existe e é diferente, resolver colisão criando sufixo
        if novo in df.columns and novo != found:
            # criar nome único
            i = 1
            novo_alt = f"{novo}_{i}"
            while novo_alt in df.columns:
                i += 1
                novo_alt = f"{novo}_{i}"


            df = df.withColumnRenamed(found, novo_alt)
            renomeados.append((found, novo_alt))
        else:
            if found == novo:
                # já está no formato desejado (não faz nada)
                renomeados.append((found, novo))
            else:
                df = df.withColumnRenamed(found, novo)
                renomeados.append((found, novo))



    return df, renomeados, nao_encontrados

In [ ]:
# 1) Substituir valores primeiro (se ainda não fez)
df = substituir_valores_df(df, map_values)   # sua função existente

# 2) Rodar renomeação robusta
df, renomeados, nao_encontrados = renomear_colunas_robusto(df, map_vars, normalize_to_snake=True, verbose=True)

# 3) Conferir rapidamente
print("Colunas atuais (preview):", df.columns[:60])
df.limit(5).show(truncate=False)


/usr/local/lib/python3.12/dist-packages/pyspark/sql/column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


Colunas atuais (preview): ['unidade_da_federacao', 'situacao_do_domicilio', 'idade_do_morador', 'sexo', 'na_semana_passada_teve_febre', 'na_semana_passada_teve_tosse', 'na_semana_passada_teve_dor_de_garganta', 'na_semana_passada_teve_dificuldade_para_respirar', 'na_semana_passada_teve_dor_de_cabeca', 'na_semana_passada_teve_dor_no_peito', 'na_semana_passada_teve_nausea', 'na_semana_passada_teve_nariz_entupido_ou_escorrendo', 'na_semana_passada_teve_fadiga', 'na_semana_passada_teve_dor_nos_olhos', 'na_semana_passada_teve_perda_de_cheiro_ou_sabor', 'na_semana_passada_teve_dor_muscular', 'por_causa_disso_foi_a_algum_estabelecimento_de_saude', 'providencia_tomada_para_recuperar_dos_sintomas_foi_ficar_em_casa', 'tem_algum_plano_de_saude_medico_seja_particular_de_empresa_ou_de_orgao_publico', 'oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus', 'qual_o_resultado', 'algum_medico_ja_lhe_deu_o_diagnostico_de_diabetes', 'algum_medico_ja_lhe_deu_o_diagnostico_de_hipertensao'

In [ ]:
# lista de colunas que indicam sintomas
colunas_agrantes = [
    "algum_medico_ja_lhe_deu_o_diagnostico_de_diabetes",
    "algum_medico_ja_lhe_deu_o_diagnostico_de_hipertensao",
    "algum_medico_ja_lhe_deu_o_diagnostico_de_asmabronquiteenfisemadoencas_respiratoria_cronica_ou_doenca_de_pulmao",
    "algum_medico_ja_lhe_deu_o_diagnostico_de_doencas_do_coracao_infarto_angina_insuficiencia_cardiaca_arritmia",
    "algum_medico_ja_lhe_deu_o_diagnostico_de_depressao",
    "algum_medico_ja_lhe_deu_o_diagnostico_de_cancer"
]

# cria expressão condicional: se QUALQUER uma for "Sim", então "Sim"
condicao_agravantes = None
for c in colunas_agrantes:
    expr = (F.col(c) == "Sim")
    condicao_agravantes = expr if condicao_agravantes is None else (condicao_agravantes | expr)

# adiciona a coluna nova
df = df.withColumn(
    "diagnostico_agravante",
    F.when(condicao_agravantes, F.lit("Sim")).otherwise(F.lit("Não"))
)



In [ ]:
# lista de colunas que indicam sintomas
colunas_sintomas = [
    "na_semana_passada_teve_febre",
    "na_semana_passada_teve_tosse",
    "na_semana_passada_teve_dor_de_garganta",
    "na_semana_passada_teve_dificuldade_para_respirar",
    "na_semana_passada_teve_dor_de_cabeca",
    "na_semana_passada_teve_dor_no_peito",
    "na_semana_passada_teve_nausea",
    "na_semana_passada_teve_nariz_entupido_ou_escorrendo",
    "na_semana_passada_teve_fadiga",
    "na_semana_passada_teve_dor_nos_olhos",
    "na_semana_passada_teve_perda_de_cheiro_ou_sabor",
    "na_semana_passada_teve_dor_muscular"
]

# cria expressão condicional: se QUALQUER uma for "Sim", então "Sim"
condicao_sintomas = None
for c in colunas_sintomas:
    expr = (F.col(c) == "Sim")
    condicao_sintomas = expr if condicao_sintomas is None else (condicao_sintomas | expr)

# adiciona a coluna nova
df = df.withColumn(
    "teve_sintomas",
    F.when(condicao_sintomas, F.lit("Sim")).otherwise(F.lit("Não"))
)



In [ ]:
df = df.withColumn(
    "classe_renda",
    F.when((F.col("valor_em_dinheiro") < 1518), "Classe E")
     .when((F.col("valor_em_dinheiro") >= 1519) & (F.col("valor_em_dinheiro") < 4554), "Classe D")
     .when((F.col("valor_em_dinheiro") >= 4554) & (F.col("valor_em_dinheiro") < 7590), "Classe C")
     .when((F.col("valor_em_dinheiro") >= 7590) & (F.col("valor_em_dinheiro") < 22270), "Classe B")
     .when((F.col("valor_em_dinheiro") > 22270) , "Classe A")
     .otherwise("Não informado")
)

In [ ]:
# dicionário estado → região
estado_para_regiao = {
    "Acre": "Norte",
    "Alagoas": "Nordeste",
    "Amapá": "Norte",
    "Amazonas": "Norte",
    "Bahia": "Nordeste",
    "Ceará": "Nordeste",
    "Distrito Federal": "Centro-Oeste",
    "Espírito Santo": "Sudeste",
    "Goiás": "Centro-Oeste",
    "Maranhão": "Nordeste",
    "Mato Grosso": "Centro-Oeste",
    "Mato Grosso do Sul": "Centro-Oeste",
    "Minas Gerais": "Sudeste",
    "Pará": "Norte",
    "Paraíba": "Nordeste",
    "Paraná": "Sul",
    "Pernambuco": "Nordeste",
    "Piauí": "Nordeste",
    "Rio de Janeiro": "Sudeste",
    "Rio Grande do Norte": "Nordeste",
    "Rio Grande do Sul": "Sul",
    "Rondônia": "Norte",
    "Roraima": "Norte",
    "Santa Catarina": "Sul",
    "São Paulo": "Sudeste",
    "Sergipe": "Nordeste",
    "Tocantins": "Norte"
}

# converte o dicionário em um map do PySpark
map_expr = F.create_map([F.lit(x) for x in sum(estado_para_regiao.items(), ())])

# cria a nova coluna com base no dicionário (mantém original se não achar no map)
df = df.withColumn(
    "regiao",
    F.coalesce(map_expr.getItem(F.col("unidade_da_federacao")), F.lit("Desconhecido"))
)


In [ ]:
df = df.withColumn(
    "faixa_etaria",
    F.when(F.col("idade_do_morador") < 14, "criança")
     .when((F.col("idade_do_morador") >= 14) & (F.col("idade_do_morador") <= 24), "adoslecente_jovem")
     .when((F.col("idade_do_morador") >= 25) & (F.col("idade_do_morador") <= 59), "adulto")
     .when(F.col("idade_do_morador") >= 60, "idoso")
     .otherwise("Não informado")
)

In [ ]:
df.show(5)

+--------------------+---------------------+----------------+------+----------------------------+----------------------------+--------------------------------------+------------------------------------------------+------------------------------------+-----------------------------------+-----------------------------+---------------------------------------------------+-----------------------------+------------------------------------+-----------------------------------------------+-----------------------------------+----------------------------------------------------+----------------------------------------------------------------+------------------------------------------------------------------------------+-----------------------------------------------------------------------+----------------+-------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------------------------------------

# Analise dos dados / manipulação e geração de bases


## consultas para validar e ajudar na analise


In [ ]:
# Registrar a view
df.createOrReplaceTempView("pnad")



In [ ]:
# Exemplo: quantos registros com idade fora do range normal
spark.sql("""
    SELECT  qual_o_resultado, count(*)
    FROM pnad
    group by qual_o_resultado
    order by qual_o_resultado
""").show()



+--------------------+--------+
|    qual_o_resultado|count(1)|
+--------------------+--------+
|                NULL|  983293|
|Ainda não recebeu...|     575|
|            Ignorado|     305|
|        Inconclusivo|      95|
|            Negativo|   64660|
|            Positivo|   26622|
+--------------------+--------+



In [ ]:
# Exemplo: quantos registros com idade fora do range normal
spark.sql("""
    SELECT  classe_renda, regiao, count(*)
    FROM pnad
    group by classe_renda, regiao
    order by classe_renda, regiao
""").show()



+------------+------------+--------+
|classe_renda|      regiao|count(1)|
+------------+------------+--------+
|    Classe A|Centro-Oeste|      25|
|    Classe A|    Nordeste|      44|
|    Classe A|       Norte|      28|
|    Classe A|     Sudeste|     140|
|    Classe A|         Sul|      89|
|    Classe B|Centro-Oeste|     274|
|    Classe B|    Nordeste|     705|
|    Classe B|       Norte|     392|
|    Classe B|     Sudeste|    2020|
|    Classe B|         Sul|    1209|
|    Classe C|Centro-Oeste|     450|
|    Classe C|    Nordeste|    1096|
|    Classe C|       Norte|     802|
|    Classe C|     Sudeste|    2552|
|    Classe C|         Sul|    1949|
|    Classe D|Centro-Oeste|    3101|
|    Classe D|    Nordeste|    7100|
|    Classe D|       Norte|    4626|
|    Classe D|     Sudeste|   16252|
|    Classe D|         Sul|   12790|
+------------+------------+--------+
only showing top 20 rows



In [ ]:
total_respostas_teve_sintomas = df.filter(F.col("teve_sintomas") == "Sim").count()


## 1. Sintomas clínicos da população

In [ ]:
#Teve algum sintoma por região
df_agregado = (
    df.groupBy("diagnostico_agravante", "teve_sintomas")
      .agg(F.count("*").alias("qtd"))
)
df_agregado.show()
#Download do arquivo para geração de gráfico em power bi
df_agregado.toPandas().to_csv("df_agregado_sintomas.csv", index=False, encoding="utf-8-sig")

files.download("df_agregado_sintomas.csv")



+---------------------+-------------+------+
|diagnostico_agravante|teve_sintomas|   qtd|
+---------------------+-------------+------+
|                  Sim|          Não|397834|
|                  Sim|          Sim| 20106|
|                  Não|          Não|637127|
|                  Não|          Sim| 20483|
+---------------------+-------------+------+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Faixa etaria x sexo x algum sintoma que teve
df_faixa_sexo = df.groupBy("faixa_etaria", "Sexo").agg(
    *[
        F.sum(F.when(F.col(c) == "Sim", 1).otherwise(0)).alias(c + "_qtd")
        for c in colunas_sintomas
    ]
)

df_faixa_sexo.show()

#Download do arquivo para geração de gráfico em power bi
df_faixa_sexo.toPandas().to_csv("df_faixa_sexo.csv", index=False, encoding="utf-8-sig")

files.download("df_faixa_sexo.csv")

+-----------------+------+--------------------------------+--------------------------------+------------------------------------------+----------------------------------------------------+----------------------------------------+---------------------------------------+---------------------------------+-------------------------------------------------------+---------------------------------+----------------------------------------+---------------------------------------------------+---------------------------------------+
|     faixa_etaria|  Sexo|na_semana_passada_teve_febre_qtd|na_semana_passada_teve_tosse_qtd|na_semana_passada_teve_dor_de_garganta_qtd|na_semana_passada_teve_dificuldade_para_respirar_qtd|na_semana_passada_teve_dor_de_cabeca_qtd|na_semana_passada_teve_dor_no_peito_qtd|na_semana_passada_teve_nausea_qtd|na_semana_passada_teve_nariz_entupido_ou_escorrendo_qtd|na_semana_passada_teve_fadiga_qtd|na_semana_passada_teve_dor_nos_olhos_qtd|na_semana_passada_teve_perda_de_cheiro_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Faixa etaria  x teve sintoma sim ou não

df_faixa_teve_sintomas = df.groupBy("faixa_etaria",  "teve_sintomas").agg(
    F.count("*").alias("qtd"))


df_faixa_teve_sintomas.show()

#Download do arquivo para geração de gráfico em power bi
df_faixa_teve_sintomas.toPandas().to_csv("df_faixa_teve_sintomas.csv", index=False, encoding="utf-8-sig")

files.download("df_faixa_teve_sintomas.csv")

+-----------------+-------------+------+
|     faixa_etaria|teve_sintomas|   qtd|
+-----------------+-------------+------+
|            idoso|          Não|181573|
|           adulto|          Não|501002|
|adoslecente_jovem|          Não|169090|
|          criança|          Não|183296|
|            idoso|          Sim|  8664|
|           adulto|          Sim| 21747|
|          criança|          Sim|  4868|
|adoslecente_jovem|          Sim|  5310|
+-----------------+-------------+------+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Porcetagem de cada sintoma das pessoas que tiveram sintomas
df_sintomas_percentual = df.agg(
    *[
        (F.sum(F.when(F.col(c) == "Sim", 1).otherwise(0)) / total_respostas_teve_sintomas * 100).alias(c + "_pct")
        for c in colunas_sintomas
    ]
)

df_sintomas_percentual.show(truncate=False)
#Download do arquivo para geração de gráfico em power bi
df_sintomas_percentual.toPandas().to_csv("df_sintomas_percentual.csv", index=False, encoding="utf-8-sig")

files.download("df_sintomas_percentual.csv")

+--------------------------------+--------------------------------+------------------------------------------+----------------------------------------------------+----------------------------------------+---------------------------------------+---------------------------------+-------------------------------------------------------+---------------------------------+----------------------------------------+---------------------------------------------------+---------------------------------------+
|na_semana_passada_teve_febre_pct|na_semana_passada_teve_tosse_pct|na_semana_passada_teve_dor_de_garganta_pct|na_semana_passada_teve_dificuldade_para_respirar_pct|na_semana_passada_teve_dor_de_cabeca_pct|na_semana_passada_teve_dor_no_peito_pct|na_semana_passada_teve_nausea_pct|na_semana_passada_teve_nariz_entupido_ou_escorrendo_pct|na_semana_passada_teve_fadiga_pct|na_semana_passada_teve_dor_nos_olhos_pct|na_semana_passada_teve_perda_de_cheiro_ou_sabor_pct|na_semana_passada_teve_dor_muscular_p

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Teve algum sintoma por região
df_agregado_regiao_sintoma = (
    df.groupBy("regiao", "teve_sintomas")
      .agg(F.count("*").alias("qtd"))
)
df_agregado_regiao_sintoma.show()

#Download do arquivo para geração de gráfico em power bi
df_agregado_regiao_sintoma.toPandas().to_csv("df_agregado_regiao_sintoma.csv", index=False, encoding="utf-8-sig")

files.download("df_agregado_regiao_sintoma.csv")


+------------+-------------+------+
|      regiao|teve_sintomas|   qtd|
+------------+-------------+------+
|         Sul|          Não|189488|
|Centro-Oeste|          Não| 47242|
|    Nordeste|          Não|337116|
|     Sudeste|          Sim| 12003|
|Centro-Oeste|          Sim|  1584|
|    Nordeste|          Sim| 13783|
|     Sudeste|          Não|324030|
|       Norte|          Sim|  5287|
|         Sul|          Sim|  7932|
|       Norte|          Não|137085|
+------------+-------------+------+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 2. Comportamento da população

In [ ]:

#Tem plano de saude x foi ao medico
df_plano_saude = (
    df.filter(F.col("tem_algum_plano_de_saude_medico_seja_particular_de_empresa_ou_de_orgao_publico").isin(["Sim", "Não"]))
      .filter(F.col("por_causa_disso_foi_a_algum_estabelecimento_de_saude").isin(["Sim", "Não"]))
      .groupBy("tem_algum_plano_de_saude_medico_seja_particular_de_empresa_ou_de_orgao_publico", "por_causa_disso_foi_a_algum_estabelecimento_de_saude")
      .agg(F.count("*").alias("qtd"))
)
df_plano_saude.show()

#Download do arquivo para geração de gráfico em power bi
df_plano_saude.toPandas().to_csv("df_plano_saude.csv", index=False, encoding="utf-8-sig")

files.download("df_plano_saude.csv")


+------------------------------------------------------------------------------+----------------------------------------------------+----+
|tem_algum_plano_de_saude_medico_seja_particular_de_empresa_ou_de_orgao_publico|por_causa_disso_foi_a_algum_estabelecimento_de_saude| qtd|
+------------------------------------------------------------------------------+----------------------------------------------------+----+
|                                                                           Sim|                                                 Não|2356|
|                                                                           Sim|                                                 Sim|1334|
|                                                                           Não|                                                 Não|7599|
|                                                                           Não|                                                 Sim|2825|
+--------------------------

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas que tiveram sintomas e fizeram o teste
df_fez_teste_sintomas= (
    df.filter(F.col("oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus").isin(["Sim", "Não"]))
      .filter(F.col("teve_sintomas") == "Sim")
      .groupBy("oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus")
      .agg(F.count("*").alias("qtd"))
      .withColumn("qtd_porcentagem", (F.col("qtd") / total_respostas_teve_sintomas * 100))
)

df_fez_teste_sintomas.show()
#Download do arquivo para geração de gráfico em power bi
df_fez_teste_sintomas.toPandas().to_csv("df_fez_teste_sintomas.csv", index=False, encoding="utf-8-sig")

files.download("df_fez_teste_sintomas.csv")

+-----------------------------------------------------------------------+-----+------------------+
|oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus|  qtd|   qtd_porcentagem|
+-----------------------------------------------------------------------+-----+------------------+
|                                                                    Não|12701|31.291729286259823|
|                                                                    Sim| 6591| 16.23838971149819|
+-----------------------------------------------------------------------+-----+------------------+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas que fizeram teste de covid por região
df_fez_teste = (
    df.filter(F.col("oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus").isin(["Sim", "Não"]))
      .groupBy("regiao", "oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus")
      .agg(F.count("*").alias("qtd"))
)
df_fez_teste.show()

#Download do arquivo para geração de gráfico em power bi
df_fez_teste.toPandas().to_csv("df_fez_teste.csv", index=False, encoding="utf-8-sig")

files.download("df_fez_teste.csv")

+------------+-----------------------------------------------------------------------+------+
|      regiao|oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus|   qtd|
+------------+-----------------------------------------------------------------------+------+
|         Sul|                                                                    Não| 64205|
|Centro-Oeste|                                                                    Não| 16804|
|    Nordeste|                                                                    Não|113757|
|     Sudeste|                                                                    Sim| 24540|
|Centro-Oeste|                                                                    Sim|  4416|
|    Nordeste|                                                                    Sim| 26630|
|     Sudeste|                                                                    Não|109619|
|       Norte|                                              

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Resultado do teste por região SWAB
df_covid_regiao = (
    df.filter(F.col("qual_o_resultado") != "Ignorado")
      .filter(F.col("qual_o_resultado") != "NULL")
      .groupBy("regiao", "qual_o_resultado" )
      .agg(F.count("*").alias("qtd"))
)
df_covid_regiao.show()

#Download do arquivo para geração de gráfico em power bi
df_covid_regiao.toPandas().to_csv("df_covid_regiao.csv", index=False, encoding="utf-8-sig")

files.download("df_covid_regiao.csv")

+------------+--------------------+-----+
|      regiao|    qual_o_resultado|  qtd|
+------------+--------------------+-----+
|     Sudeste|Ainda não recebeu...|  213|
|       Norte|Ainda não recebeu...|   34|
|       Norte|            Negativo| 8956|
|Centro-Oeste|            Negativo| 3533|
|     Sudeste|            Positivo| 8369|
|Centro-Oeste|        Inconclusivo|    3|
|     Sudeste|        Inconclusivo|   37|
|       Norte|            Positivo| 3670|
|Centro-Oeste|Ainda não recebeu...|   17|
|    Nordeste|            Positivo| 9413|
|     Sudeste|            Negativo|20593|
|Centro-Oeste|            Positivo| 1348|
|         Sul|Ainda não recebeu...|  172|
|    Nordeste|Ainda não recebeu...|  139|
|       Norte|        Inconclusivo|   15|
|         Sul|        Inconclusivo|   19|
|         Sul|            Negativo|12819|
|         Sul|            Positivo| 3822|
|    Nordeste|        Inconclusivo|   21|
|    Nordeste|            Negativo|18759|
+------------+--------------------

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas que tiveram sintomas se procuraram medico ou não

df_foi_ao_medico = (
    df.filter(F.col("por_causa_disso_foi_a_algum_estabelecimento_de_saude").isin(["Sim", "Não"]))
      .filter(F.col("teve_sintomas") == "Sim")
      .groupBy("por_causa_disso_foi_a_algum_estabelecimento_de_saude")
      .agg(F.count("*").alias("qtd"))
      .withColumn("qtd_porcentagem", (F.col("qtd") / total_respostas_teve_sintomas * 100))
)

df_foi_ao_medico.show()

#Download do arquivo para geração de gráfico em power bi
df_foi_ao_medico.toPandas().to_csv("df_foi_ao_medico.csv", index=False, encoding="utf-8-sig")

files.download("df_foi_ao_medico.csv")


+----------------------------------------------------+-----+------------------+
|por_causa_disso_foi_a_algum_estabelecimento_de_saude|  qtd|   qtd_porcentagem|
+----------------------------------------------------+-----+------------------+
|                                                 Não|17163|42.284855502722415|
|                                                 Sim|16305| 40.17098228584099|
+----------------------------------------------------+-----+------------------+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas que tiveram sintomas e ficaram em casa

df_ficou_em_casa= (
    df.filter(F.col("providencia_tomada_para_recuperar_dos_sintomas_foi_ficar_em_casa").isin(["Sim", "Não"]))
      .filter(F.col("teve_sintomas") == "Sim")
      .groupBy("providencia_tomada_para_recuperar_dos_sintomas_foi_ficar_em_casa")
      .agg(F.count("*").alias("qtd"))
      .withColumn("qtd_porcentagem", (F.col("qtd") / total_respostas_teve_sintomas * 100))
)

df_ficou_em_casa.show()

#Download do arquivo para geração de gráfico em power bi
df_ficou_em_casa.toPandas().to_csv("df_ficou_em_casa.csv", index=False, encoding="utf-8-sig")

files.download("df_ficou_em_casa.csv")

+----------------------------------------------------------------+-----+-----------------+
|providencia_tomada_para_recuperar_dos_sintomas_foi_ficar_em_casa|  qtd|  qtd_porcentagem|
+----------------------------------------------------------------+-----+-----------------+
|                                                             Não|20564|50.66397299761019|
|                                                             Sim| 9159|22.56522703195447|
+----------------------------------------------------------------+-----+-----------------+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas que tiveram sintomas e não fizeram o teste divido por região

df_nao_fez_teste_regiao= (
    df.filter(F.col("oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus") == "Não")
      .filter(F.col("teve_sintomas") == "Sim")
      .groupBy("regiao")
      .agg(F.count("*").alias("qtd"))

)

df_nao_fez_teste_regiao.show()

#Download do arquivo para geração de gráfico em power bi
df_nao_fez_teste_regiao.toPandas().to_csv("df_nao_fez_teste_regiao.csv", index=False, encoding="utf-8-sig")

files.download("df_nao_fez_teste_regiao.csv")

+------------+----+
|      regiao| qtd|
+------------+----+
|    Nordeste|4221|
|         Sul|2497|
|     Sudeste|4034|
|Centro-Oeste| 457|
|       Norte|1492|
+------------+----+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas que tiveram sintomas o que fizeram?

df_teve_sintomas_medida_restricao= (
    df.filter(F.col("teve_sintomas") == "Sim")
      .groupBy("qual_foi_o_resultado_do_teste_na_semana_passada_devido_a_pandemia_do_coronavirus_em_que_medida_oa_sra_restringiu_o_contato_com_as_pessoas")
      .agg(F.count("*").alias("qtd"))

)

df_teve_sintomas_medida_restricao.show()

#Download do arquivo para geração de gráfico em power bi
df_teve_sintomas_medida_restricao.toPandas().to_csv("df_teve_sintomas_medida_restricao.csv", index=False, encoding="utf-8-sig")

files.download("df_teve_sintomas_medida_restricao.csv")

+-----------------------------------------------------------------------------------------------------------------------------------------+-----+
|qual_foi_o_resultado_do_teste_na_semana_passada_devido_a_pandemia_do_coronavirus_em_que_medida_oa_sra_restringiu_o_contato_com_as_pessoas|  qtd|
+-----------------------------------------------------------------------------------------------------------------------------------------+-----+
|                                                                                                                     Ficou rigorosamen...| 2477|
|                                                                                                                                 Ignorado|   12|
|                                                                                                                     Reduziu o contato...| 5072|
|                                                                                                                     Ficou 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas que tiveram sintomas o que fizeram por regiao?

df_teve_sintomas_medida_restricao_regiao= (
    df.filter(F.col("teve_sintomas") == "Sim")
      .filter(F.col("qual_foi_o_resultado_do_teste_na_semana_passada_devido_a_pandemia_do_coronavirus_em_que_medida_oa_sra_restringiu_o_contato_com_as_pessoas") != "Ignorado")
      .groupBy("qual_foi_o_resultado_do_teste_na_semana_passada_devido_a_pandemia_do_coronavirus_em_que_medida_oa_sra_restringiu_o_contato_com_as_pessoas", "regiao")
      .agg(F.count("*").alias("qtd"))

)

df_teve_sintomas_medida_restricao_regiao.show()


#Download do arquivo para geração de gráfico em power bi
df_teve_sintomas_medida_restricao_regiao.toPandas().to_csv("df_teve_sintomas_medida_restricao_regiao.csv", index=False, encoding="utf-8-sig")

files.download("df_teve_sintomas_medida_restricao_regiao.csv")


+-----------------------------------------------------------------------------------------------------------------------------------------+------------+----+
|qual_foi_o_resultado_do_teste_na_semana_passada_devido_a_pandemia_do_coronavirus_em_que_medida_oa_sra_restringiu_o_contato_com_as_pessoas|      regiao| qtd|
+-----------------------------------------------------------------------------------------------------------------------------------------+------------+----+
|                                                                                                                     Ficou rigorosamen...|         Sul| 592|
|                                                                                                                     Reduziu o contato...|    Nordeste|1565|
|                                                                                                                     Ficou em casa e s...|         Sul| 929|
|                                                   

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 3. Características econômicas da sociedade

In [ ]:
#quantidade de pessoas urbana x rural
df_situacao_do_domicilio = (
    df.groupBy("situacao_do_domicilio")
      .agg(F.count("*").alias("qtd"))
)
df_situacao_do_domicilio.show()

#Download do arquivo para geração de gráfico em power bi
df_situacao_do_domicilio.toPandas().to_csv("df_situacao_do_domicilio.csv", index=False, encoding="utf-8-sig")

files.download("df_situacao_do_domicilio.csv")

+---------------------+------+
|situacao_do_domicilio|   qtd|
+---------------------+------+
|               Urbana|812722|
|                Rural|262828|
+---------------------+------+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas que fizeram teste de covid por classe de renda

df_fez_teste_classe_renda = (
    df.filter(F.col("oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus").isin(["Sim", "Não"]))
      .filter(F.col("classe_renda") != "Não informado")
      .groupBy("classe_renda", "oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus")
      .agg(F.count("*").alias("qtd"))
)

df_fez_teste_classe_renda.show()

#Download do arquivo para geração de gráfico em power bi
df_fez_teste_classe_renda.toPandas().to_csv("df_fez_teste_classe_renda.csv", index=False, encoding="utf-8-sig")

files.download("df_fez_teste_classe_renda.csv")

+------------+-----------------------------------------------------------------------+-----+
|classe_renda|oa_sra_fez_algum_teste_para_saber_se_estava_infectadoa_pelo_coronavirus|  qtd|
+------------+-----------------------------------------------------------------------+-----+
|    Classe E|                                                                    Sim|11976|
|    Classe C|                                                                    Sim| 2131|
|    Classe B|                                                                    Não| 2732|
|    Classe B|                                                                    Sim| 1829|
|    Classe D|                                                                    Não|33577|
|    Classe C|                                                                    Não| 4663|
|    Classe A|                                                                    Não|  166|
|    Classe D|                                                        

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas que pediram emprestimo
df_pediu_emprestimo = (
    df.filter(
          (F.col("durante_o_periodo_da_pandemia_alguem_deste_domicilio_solicitou_algum_emprestimo").like("Sim%")) |
          (F.col("durante_o_periodo_da_pandemia_alguem_deste_domicilio_solicitou_algum_emprestimo").like("Não%"))
      )
      .groupBy("durante_o_periodo_da_pandemia_alguem_deste_domicilio_solicitou_algum_emprestimo")
      .agg(F.count("*").alias("qtd"))
)
df_pediu_emprestimo.show()

#Download do arquivo para geração de gráfico em power bi
df_pediu_emprestimo.toPandas().to_csv("df_pediu_emprestimo.csv", index=False, encoding="utf-8-sig")

files.download("df_pediu_emprestimo.csv")

+-------------------------------------------------------------------------------+------+
|durante_o_periodo_da_pandemia_alguem_deste_domicilio_solicitou_algum_emprestimo|   qtd|
+-------------------------------------------------------------------------------+------+
|                                                           Sim, e pelo menos...| 29574|
|                                                           Sim, mas nenhum m...| 53674|
|                                                                  Não solicitou|325037|
+-------------------------------------------------------------------------------+------+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas que pediram emprestimo por região
df_pediu_emprestimo_regiao = (
    df.filter(
          (F.col("durante_o_periodo_da_pandemia_alguem_deste_domicilio_solicitou_algum_emprestimo").like("Sim%")) |
          (F.col("durante_o_periodo_da_pandemia_alguem_deste_domicilio_solicitou_algum_emprestimo").like("Não%"))
      )
      .groupBy("durante_o_periodo_da_pandemia_alguem_deste_domicilio_solicitou_algum_emprestimo", "regiao")
      .agg(F.count("*").alias("qtd"))
)
df_pediu_emprestimo_regiao.show()

#Download do arquivo para geração de gráfico em power bi
df_pediu_emprestimo_regiao.toPandas().to_csv("df_pediu_emprestimo_regiao.csv", index=False, encoding="utf-8-sig")

files.download("df_pediu_emprestimo_regiao.csv")

+-------------------------------------------------------------------------------+------------+------+
|durante_o_periodo_da_pandemia_alguem_deste_domicilio_solicitou_algum_emprestimo|      regiao|   qtd|
+-------------------------------------------------------------------------------+------------+------+
|                                                           Sim, e pelo menos...|     Sudeste|  8789|
|                                                           Sim, mas nenhum m...|       Norte|  5906|
|                                                                  Não solicitou|         Sul| 58299|
|                                                                  Não solicitou|Centro-Oeste| 15719|
|                                                           Sim, mas nenhum m...|         Sul| 11672|
|                                                           Sim, mas nenhum m...|Centro-Oeste|  2639|
|                                                           Sim, e pelo menos...| 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas precisaram auxilio
df_precisou_aux = (
    df.filter(F.col("auxilios_emergenciais_relacionados_ao_coronavirus").isin(["Sim", "Não"]))
      .groupBy("auxilios_emergenciais_relacionados_ao_coronavirus")
      .agg(F.count("*").alias("qtd"))
)
df_precisou_aux.show()


#Download do arquivo para geração de gráfico em power bi
df_precisou_aux.toPandas().to_csv("df_precisou_aux.csv", index=False, encoding="utf-8-sig")

files.download("df_precisou_aux.csv")

+-------------------------------------------------+------+
|auxilios_emergenciais_relacionados_ao_coronavirus|   qtd|
+-------------------------------------------------+------+
|                                              Não|183093|
|                                              Sim|174856|
+-------------------------------------------------+------+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#quantidade de pessoas precisaram auxilio pore região
df_precisou_aux_regiao = (
    df.filter(F.col("auxilios_emergenciais_relacionados_ao_coronavirus").isin(["Sim", "Não"]))
      .groupBy("auxilios_emergenciais_relacionados_ao_coronavirus", "regiao")
      .agg(F.count("*").alias("qtd"))
)
df_precisou_aux_regiao.show()

#Download do arquivo para geração de gráfico em power bi
df_precisou_aux_regiao.toPandas().to_csv("df_precisou_aux_regiao.csv", index=False, encoding="utf-8-sig")

files.download("df_precisou_aux_regiao.csv")

+-------------------------------------------------+------------+-----+
|auxilios_emergenciais_relacionados_ao_coronavirus|      regiao|  qtd|
+-------------------------------------------------+------------+-----+
|                                              Sim|    Nordeste|72505|
|                                              Sim|         Sul|19848|
|                                              Não|    Nordeste|43826|
|                                              Não|     Sudeste|64690|
|                                              Sim|     Sudeste|46681|
|                                              Não|       Norte|19434|
|                                              Não|         Sul|45387|
|                                              Não|Centro-Oeste| 9756|
|                                              Sim|       Norte|28061|
|                                              Sim|Centro-Oeste| 7761|
+-------------------------------------------------+------------+-----+



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>